In [96]:
import pandas as pd
import re
from pymystem3 import Mystem
import many_stop_words
ru_stopwords = many_stop_words.get_stop_words('ru').union({'либо', 'б', 'срок', 'иной', 'преступление', 'наказание', 'мера',
                                                           'лишение', 'свобода', 'гражданин', 'размер', 'лицо'})

In [97]:
df = pd.read_csv('articles.csv', index_col=0)

In [98]:
sec = {'I':1, 'II':2, 'III':3, 'IV':4, 'V':5, 'VI':6, 'VII':7, 'VIII':8, 'IX':9, 'X':10, 'XI':11, 'XII':11}
df['section'] = df.section.map(lambda x: sec[x])

In [99]:
pattern = re.compile('\s\((\w|\s)*стать.*?\)')
df['parsed_body'] = df['body'].map(lambda x: re.sub(pattern, '', x))

In [100]:
m = Mystem(entire_input=False)

In [101]:
%%time
df['lemmas'] = df['parsed_body'].map(m.lemmatize)

CPU times: user 621 ms, sys: 111 ms, total: 731 ms
Wall time: 8.16 s


{chapter} |Hierarchy number:{number} section:{section} |{lemmas} |NameFeatures SectionName:{section_name} ChapterName:{chapter_name} Name:{name}

In [102]:
df['lemmas'] = df['lemmas'].map(lambda lemmas: list(filter(lambda lemma: lemma not in ru_stopwords, lemmas)))

In [103]:
vw_text_form = 'article{number:.0f} {text}' 
# 
to_vw = lambda row: vw_text_form.format(
    text=' '.join(row['lemmas']),
    **row
)

In [104]:
df['vw'] = df.apply(to_vw,axis=1)

In [105]:
df[['vw']].to_csv('vw.txt', index=False, header=False)